In [571]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
from math import sqrt
import time

In [572]:
# create ratings.dat dictionary
ratings = open("./ml-1m/ratings.dat","r")
userRatingsDict = {}

for rating in ratings:
    data = rating.split('::')
    userID = data[0]
    movieID = data[1]
    rating = int(data[2])
    # setup movie ratings to same userID
    userRatingsDict.setdefault(userID,{})
    temp = userRatingsDict[userID]
    temp.update({movieID: rating})
    userRatingsDict[userID].update(temp)

print(userRatingsDict['1'])
ratings.close()

{'1193': 5, '661': 3, '914': 3, '3408': 4, '2355': 5, '1197': 3, '1287': 5, '2804': 5, '594': 4, '919': 4, '595': 5, '938': 4, '2398': 4, '2918': 4, '1035': 5, '2791': 4, '2687': 3, '2018': 4, '3105': 5, '2797': 4, '2321': 3, '720': 3, '1270': 5, '527': 5, '2340': 3, '48': 5, '1097': 4, '1721': 4, '1545': 4, '745': 3, '2294': 4, '3186': 4, '1566': 4, '588': 4, '1907': 4, '783': 4, '1836': 5, '1022': 5, '2762': 4, '150': 5, '1': 5, '1961': 5, '1962': 4, '2692': 4, '260': 4, '1028': 5, '1029': 5, '1207': 4, '2028': 5, '531': 4, '3114': 4, '608': 4, '1246': 4}


In [573]:
# create users.dat dictionary
users = open("./ml-1m/users.dat","r")
usersDict = {}

for user in users:
    data = user.split('::')
    userID = data[0]
    gender = data[1]
    age = int(data[2])
    occupation = int(data[3])
    #setup user
    usersDict[userID] = {'gender': gender, 'age': age, 'occupation': occupation}

print(usersDict['1'])
users.close()

{'gender': 'F', 'age': 1, 'occupation': 10}


In [574]:
# create movies.dat dictionary
movies = open("./ml-1m/movies.dat","r",encoding='iso-8859-15')
moviesDict = {}

for movie in movies:
    data = movie.split('::')
    movieID = data[0]
    title = data[1]
    genres = data[2]
    #setup movie
    moviesDict[movieID] = {'movieID': movieID, 'title': title, 'genres': genres}

print(moviesDict['1'])
movies.close()

{'movieID': '1', 'title': 'Toy Story (1995)', 'genres': "Animation|Children's|Comedy\n"}


In [575]:
# predictor data
knowData ="""1::Toy Story (1995)::Animation|Children's|Comedy::5
2::Jumanji (1995)::Adventure|Children's|Fantasy::4
9::Sudden Death (1995)::Action::2
10::GoldenEye (1995)::Action|Adventure|Thriller::2
13::Balto (1995)::Animation|Children's::1
14::Nixon (1995)::Drama::1
17::Sense and Sensibility (1995)::Drama|Romance::1
23::Assassins (1995)::Thriller::3
47::Seven (Se7en) (1995)::Crime|Thriller::2
356::Forrest Gump (1994)::Comedy|Romance|War::5
3147::Green Mile, The (1999)::Drama|Thriller::5
593::Silence of the Lambs, The (1991)::Drama|Thriller::2
2028::Saving Private Ryan (1998)::Action|Drama|War::5
838::Emma (1996)::Comedy|Drama|Romance::1
1721::Titanic (1997)::Drama|Romance::5
2628::Star Wars: Episode I - The Phantom Menace (1999)::Action|Adventure|Fantasy|Sci-Fi::4
1608::Air Force One (1997)::Action|Thriller::4
165::Die Hard: With a Vengeance (1995)::Action|Thriller::4
589::Terminator 2: Judgment Day (1991)::Action|Sci-Fi|Thriller::2"""

unknowData = """318::Shawshank Redemption, The (1994)::Drama
527::Schindler's List (1993)::Drama|War
2959::Fight Club (1999)::Drama
393::Street Fighter (1994)::Action
3285::Beach, The (2000)::Adventure|Drama
2571::Matrix, The (1999)::Action|Sci-Fi|Thriller
1270::Back to the Future (1985)::Comedy|Sci-Fi
3578::Gladiator (2000)::Action|Drama
1200::Aliens (1986)::Action|Sci-Fi|Thriller|War
2858::American Beauty (1999)::Comedy|Drama
22::Copycat (1995)::Crime|Drama|Thriller"""

In [576]:
# put predictor know data into dictionary
knowDataDict = {}
unknowDataDict = {}
totalDataDict = {}
for rating in knowData.split("\n"):
    data = rating.split("::")
    movieID = data[0]
    rating = int(data[3])
    knowDataDict[movieID] = rating
    totalDataDict[movieID] = rating

for rating in unknowData.split("\n"):
    data = rating.split("::")
    movieID = data[0]
    unknowDataDict.setdefault(movieID, 0)
    totalDataDict.setdefault(movieID, 0)

userRatingsDict['predictor'] = knowDataDict
userRatingsDict['unpredictor'] = unknowDataDict
userRatingsDict['predictorTotal'] = totalDataDict

print(userRatingsDict['unpredictor'])

{'318': 0, '527': 0, '2959': 0, '393': 0, '3285': 0, '2571': 0, '1270': 0, '3578': 0, '1200': 0, '2858': 0, '22': 0}


In [577]:
# put predictor unknow data into dictionary
unknowDataArray = []
for rating in unknowData.split("\n"):
    data = rating.split("::")
    movieID = data[0]
    unknowDataArray.append(movieID)

print(unknowDataArray)

['318', '527', '2959', '393', '3285', '2571', '1270', '3578', '1200', '2858', '22']


In [578]:
def sim_distance(prefs, p1, p2):
    '''
    Returns a distance-based similarity score for person1 and person2.
    '''

    # Get the list of shared_items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they have no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[p1][item] - prefs[p2][item], 2) for item in prefs[p1] if item in prefs[p2]])
    return 1 / (1 + sqrt(sum_of_squares))

In [579]:
sim_distance(userRatingsDict,"predictor","674")

0.2612038749637414

In [580]:
def sim_pearson(prefs, p1, p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2.
    '''

    # Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they are no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Sum calculations
    n = len(si)
    # Sums of all the preferences
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])
    # Sums of the squares
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])
    # Sum of the products
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    r = num / den
    return r

In [581]:
sim_pearson(userRatingsDict,'predictor','674')

1.0000000000000018

In [582]:
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    '''
    Returns the best matches for person from the prefs dictionary. 
    Number of results and similarity function are optional params.
    '''

    scores = [(similarity(prefs, person, other), other) for other in prefs
              if other != person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [583]:
topMatches(userRatingsDict,"predictor", 10)

[(1.000000000000004, '5289'),
 (1.0000000000000033, '6026'),
 (1.0000000000000033, '2742'),
 (1.0000000000000022, '379'),
 (1.0000000000000022, '3585'),
 (1.0000000000000022, '2622'),
 (1.0000000000000018, '674'),
 (1.0000000000000018, '5662'),
 (1.0000000000000018, '5293'),
 (1.0000000000000018, '4657')]

In [584]:
def getRecommendations(prefs, person, similarity=sim_pearson):
    '''
    Gets recommendations for a person by using a weighted average
    of every other user's rankings
    '''

    totals = {}
    simSums = {}
    for other in prefs:
    # Don't compare me to myself
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        # Ignore scores of zero or lower
        if sim <= 0:
            continue
        for item in prefs[other]:
            # Only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item] == 0:
                # Similarity * Score
                totals.setdefault(item, 0)
                # The final score is calculated by multiplying each item by the
                #   similarity and adding these products together
                totals[item] += prefs[other][item] * sim
                # Sum of similarities
                simSums.setdefault(item, 0)
                simSums[item] += sim
    # Create the normalized list
#     rankings = [(total / simSums[item], item) for (item, total) in
#                 totals.items()]
    rankings = [(item, total / simSums[item]) for (item, total) in
                totals.items()]
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [585]:
# get unknow movies rakings
# calculator user base process time
userBaseStartTime = time.time()
recommendations = getRecommendations(userRatingsDict,"predictor")
userBaseProcessTime = time.time() - userBaseStartTime

recommendationsDict = {}
for recommendation in recommendations:
    movieID = recommendation[0]
    rakings = recommendation[1]
    recommendationsDict[movieID] = rakings
    
for moviesID in unknowDataArray :
    print(moviesID)
    print(recommendationsDict[moviesID])


318
4.617450650542291
527
4.551936980692199
2959
3.9344022244718455
393
1.9839770864961601
3285
2.850910844821836
2571
4.23958050507157
1270
4.064846703342833
3578
4.138827675012243
1200
3.968986148390578
2858
4.2448011367183085
22
3.2499542072144365


In [586]:
def transformPrefs(prefs):
    '''
    Transform the recommendations into a mapping where persons are described
    with interest scores for a given title e.g. {title: person} instead of
    {person: title}.
    '''

    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # Flip item and person
            result[item][person] = prefs[person][item]
    return result

In [587]:
def calculateSimilarItems(prefs, n=10):
    '''
    Create a dictionary of items showing which other items they are
    most similar to.
    '''

    result = {}
    # Invert the preference matrix to be item-centric
    itemPrefs = transformPrefs(prefs)
    c = 0
    for item in itemPrefs:
        # Find the most similar items to this one
        similar_item = topMatches(userRatingsDict, item, n=n, similarity=sim_distance)
        result[item] = similar_item
    return result

In [588]:
predictorDict = {}
predictorDict['predictorTotal'] = userRatingsDict['predictorTotal']
similarItemsResult = calculateSimilarItems(predictorDict, 10)
for i in similarItemsResult.keys():
    print (i, similarItemsResult[i])

1 [(1.0, '994'), (1.0, '986'), (1.0, '978'), (1.0, '961'), (1.0, '906'), (1.0, '901'), (1.0, '841'), (1.0, '827'), (1.0, '797'), (1.0, '734')]
2 [(1.0, '902'), (1.0, '89'), (1.0, '784'), (1.0, '745'), (1.0, '709'), (1.0, '681'), (1.0, '640'), (1.0, '606'), (1.0, '5993'), (1.0, '5897')]
9 [(1.0, '892'), (1.0, '6012'), (1.0, '5983'), (1.0, '5879'), (1.0, '5802'), (1.0, '5729'), (1.0, '5577'), (1.0, '5564'), (1.0, '5467'), (1.0, '5417')]
10 [(1.0, '3043'), (1.0, '2037'), (0.5, '61'), (0.5, '4421'), (0.5, '421'), (0.5, '356'), (0.5, '3133'), (0.5, '2819'), (0.5, '2710'), (0.5, '2488')]
13 [(1.0, '941'), (1.0, '903'), (1.0, '89'), (1.0, '874'), (1.0, '870'), (1.0, '835'), (1.0, '794'), (1.0, '741'), (1.0, '704'), (1.0, '684')]
14 [(1.0, '994'), (1.0, '982'), (1.0, '979'), (1.0, '946'), (1.0, '933'), (1.0, '931'), (1.0, '868'), (1.0, '853'), (1.0, '820'), (1.0, '776')]
17 [(1.0, '942'), (1.0, '821'), (1.0, '766'), (1.0, '5215'), (1.0, '5125'), (1.0, '5067'), (1.0, '4971'), (1.0, '4755'), (1.

In [589]:
def getRecommendedItems_item_based(prefs, similar_item_list, user):
    userRatings = prefs[user]
    scores = {}
    totalSim = {}
    # Loop over items rated by this user
    for (item, rating) in userRatings.items():
        if item not in similar_item_list:
            continue
        # Loop over items similar to this one
        for (similarity, item2) in similar_item_list[item]:
            # Ignore if this user has already rated this item
            if item2 in userRatings:
                continue
            # Weighted sum of rating times similarity
            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating
            # Sum of all the similarities
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity
    # Divide each total score by total weighting to get an average
    rankings = [(score / totalSim[item], item) for (item, score) in
                scores.items()]
    # Return the rankings from highest to lowest
    rankings.sort()
    rankings.reverse()
    return rankings

In [590]:
# get item base movies
# calculator item base process time
itemBaseStartTime = time.time()
result = getRecommendedItems_item_based(userRatingsDict, similarItemsResult, 'predictor')
itemBaseProcessTime = time.time() - itemBaseStartTime
print(result)

[(5.0, '997'), (5.0, '991'), (5.0, '986'), (5.0, '978'), (5.0, '965'), (5.0, '961'), (5.0, '959'), (5.0, '951'), (5.0, '939'), (5.0, '932'), (5.0, '923'), (5.0, '916'), (5.0, '906'), (5.0, '901'), (5.0, '894'), (5.0, '886'), (5.0, '852'), (5.0, '841'), (5.0, '827'), (5.0, '823'), (5.0, '812'), (5.0, '799'), (5.0, '797'), (5.0, '775'), (5.0, '734'), (5.0, '717'), (5.0, '703'), (5.0, '612'), (5.0, '5999'), (5.0, '5945'), (5.0, '489'), (5.0, '4723'), (5.0, '447'), (4.5, '745'), (4.5, '683'), (4.0, '989'), (4.0, '94'), (4.0, '917'), (4.0, '902'), (4.0, '878'), (4.0, '871'), (4.0, '834'), (4.0, '833'), (4.0, '784'), (4.0, '769'), (4.0, '759'), (4.0, '719'), (4.0, '709'), (4.0, '681'), (4.0, '668'), (4.0, '606'), (4.0, '6020'), (4.0, '5993'), (4.0, '5989'), (4.0, '5897'), (4.0, '5839'), (4.0, '5760'), (4.0, '5725'), (4.0, '5642'), (4.0, '5258'), (3.6666666666666665, '994'), (3.6666666666666665, '892'), (3.6666666666666665, '5462'), (3.3333333333333335, '874'), (3.0, '942'), (3.0, '89'), (3.0

In [591]:
# get 45 years old woman movies dictionary
womanRatingsDict = {}

for userID, data in usersDict.items():
    gender = data['gender']
    age = data['age']
    if gender == 'F' and age == 45:
        womanRatingsDict[userID] = userRatingsDict[userID]

In [592]:
# get one user movies average rating
def getUserAverageRating(prefs, user):
    return sum([rating for (movieID, rating) in prefs.items()]) / len(prefs)

In [593]:
# get movie's rating and count by moviesID
def getHigherThanAverageRatingMovies(prefs):
    womanMoviesCount = {}
    for (userID, data) in prefs.items():
        averageRating = getUserAverageRating(data, userID)
        for (movieID, rating) in data.items():
            if rating > averageRating:
                if movieID not in womanMoviesCount:
                    womanMoviesCount[movieID] = {'score': rating, 'count': 1}
                else:
                    data = womanMoviesCount[movieID]
                    score = data['score'] + rating
                    count = data['count'] + 1
                    avger = score / count
                    womanMoviesCount[movieID] = {'score': score, 'count': count}
    return womanMoviesCount

In [594]:
# get top count movies 
def getTopCountMovies(prefs, n=5):
    womanCountMovies = {}
    for (movieID, data) in prefs.items():
        score = data['score']
        count = data['count']
        womanCountMovies[movieID] = count
    
    ratings = [(rating, item) for (item, rating) in womanCountMovies.items()]
    ratings.sort()
    ratings.reverse()
    return ratings[0:n]

In [595]:
# get top average movies 
def getTopScoreMovies(prefs, n=5):
    womanScoreMovies = {}
    for (movieID, data) in prefs.items():
        score = data['score']
        count = data['count']
        womanScoreMovies[movieID] = score
    
    ratings = [(score, item) for (item, score) in womanScoreMovies.items()]
    ratings.sort()
    ratings.reverse()
    return ratings[0:n]

In [596]:
def convertDictionaryByList(items):
    resultDict = {} 
    for item in items:
        resultDict[item[1]] = item[0]
    return resultDict

In [597]:
def getTopAverageMovies(topCount, topScore):
    womanAverageMovies = {}
    for (movie, count) in topCount.items():
        movieID = str(movie)
        if movieID in topScore:
            score = topScore[movieID]
            womanAverageMovies[movieID] = score / count
            
    ratings = [(avger, item) for (item, avger) in womanAverageMovies.items()]
    ratings.sort()
    ratings.reverse()
    return ratings

In [598]:
womanMoviesDict = getHigherThanAverageRatingMovies(womanRatingsDict)
# get list
womanTopCount = getTopCountMovies(womanMoviesDict, 10)
womanTopScore = getTopScoreMovies(womanMoviesDict, 10)
# conver to dictionary
womanTopCountDict = convertDictionaryByList(womanTopCount)
womanTopScoreDict = convertDictionaryByList(womanTopScore)
# get average score
womanTopAverage = getTopAverageMovies(womanTopCountDict, womanTopScoreDict)
print('womanTopCountDict')
print(womanTopCountDict)
print('womanTopScoreDict')
print(womanTopScoreDict)
print('womanTopAverage')
print(womanTopAverage)

womanTopCountDict
{'527': 67, '2858': 62, '2396': 61, '260': 55, '318': 54, '593': 50, '2762': 49, '912': 48, '34': 47, '356': 46}
womanTopScoreDict
{'527': 323, '2858': 292, '2396': 282, '318': 255, '260': 255, '2762': 229, '912': 227, '593': 227, '34': 214, '2028': 214}
womanTopAverage
[(4.82089552238806, '527'), (4.729166666666667, '912'), (4.722222222222222, '318'), (4.709677419354839, '2858'), (4.673469387755102, '2762'), (4.636363636363637, '260'), (4.622950819672131, '2396'), (4.553191489361702, '34'), (4.54, '593')]


In [599]:
#show top average movies
for data in womanTopAverage:
    movie = moviesDict[data[1]]
    print(movie['movieID'] + ', ' + movie['title'])
    print(data[0])

527, Schindler's List (1993)
4.82089552238806
912, Casablanca (1942)
4.729166666666667
318, Shawshank Redemption, The (1994)
4.722222222222222
2858, American Beauty (1999)
4.709677419354839
2762, Sixth Sense, The (1999)
4.673469387755102
260, Star Wars: Episode IV - A New Hope (1977)
4.636363636363637
2396, Shakespeare in Love (1998)
4.622950819672131
34, Babe (1995)
4.553191489361702
593, Silence of the Lambs, The (1991)
4.54


In [600]:
# get similar movies of Matrix(2517)
topMovies = topMatches(transformPrefs(userRatingsDict) , '2517', 10)
movies = [(movieID, rating) for (rating, movieID) in topMovies]
for (movieID, rating) in movies:
    movie = moviesDict[movieID]
    print(movie['movieID'] + ', ' + movie['title'])
    print(rating)


659, Purple Noon (1960)
1.0000000000000027
3302, Beautiful People (1999)
1.0000000000000027
3283, Minnie and Moskowitz (1971)
1.0000000000000007
3027, Slaughterhouse 2 (1988)
1.0000000000000007
967, Outlaw, The (1943)
1.0
962, They Made Me a Criminal (1939)
1.0
958, Lady of Burlesque (1943)
1.0
769, Marlene Dietrich: Shadow and Light (1996)
1.0
754, Gold Diggers: The Secret of Bear Mountain (1995)
1.0
696, Butterfly Kiss (1995)
1.0


In [601]:
print(userBaseProcessTime)
print(itemBaseProcessTime)

0.3680899143218994
0.00047516822814941406
